In [1]:
import os
import pandas as pd
import h5py
import numpy

#Load the Excel file of Kene's dataset
excel_file_path = 'SignPostageData/signage_data.xlsx'

#Read it using pandas
df = pd.read_excel(excel_file_path)

#Split the data into the main dataset and metadata
sign_location_data = df.iloc[:42]  # Data rows (1 to 42 inclusive)
metadata_df = df.iloc[45:48]  # Metadata rows (46 to 48)


#--------------------------------------TO FIX MANY FORMAT ISSUES I ENCOUNTERED---------------------------------

#Clean the data by removing non-breaking spaces and unwanted smart quotes
def clean_text(text):
    if isinstance(text, str):
        text = text.replace(u'\xa0', ' ') # Replace non-breaking spaces with normal spaces
        text = text.replace(u'\u2019', "'") # Replace smart single quote ’ with straight '
        text = text.replace(u'\u201d', '"') # Replace smart double quote ” with straight "
        text = text.replace(u'\u2018', "'") # Replace left single quote with straight '
        text = text.replace(u'\u201c', '"') # Replace left double quote with straight "
        text = text.replace(u'\xc2\xb0', '°') # Properly encode the degree symbol
        text = text.replace(u'\xc2\xb4', "'")  # Replace byte-encoded acute accent with plain apostrophe
    return text
    
#Apply cleaning to the whole dataframe
sign_location_data = sign_location_data.map(clean_text)

#Check for missing or incompatible data types and handle them-NaN -> empty strings or 0 for numeric values
sign_location_data = sign_location_data.fillna('')  
metadata_df = metadata_df.fillna('')  

# Convert all columns to string type to ensure they are stored as readable text
sign_location_data = sign_location_data.astype(str)

for column in sign_location_data.columns:
    if sign_location_data[column].dtype == 'O':  #'O' stands for object
        sign_location_data[column] = sign_location_data[column].astype(str)  #Convert object types to strings

#Convert the DataFrame to a list of lists
sign_location_data_list = sign_location_data.values.tolist()


#-------------------------------------------------EXTRACT_METADATA----------------------------------------------

#print(metadata_df)
metadata_row_1 = df.loc[44, 'Location'] #about the dataset
metadata_row_2 = df.loc[45, 'Location'] #about the collection tool
metadata_row_3 = df.loc[46, 'Location'] #about the latitude/longitude formaT

# Convert metadata to a dictionary
metadata_dict = {
    "Dataset_Description": metadata_row_1,
    "Collection_Tool": metadata_row_2,
    "Latitude_Longitude_Format": metadata_row_3
}

#---------------------------------------------CREATE_HDF5_FILE--------------------------------------------------

#Create the HDF5 file
hdf5_file_name = 'SignLocations.h5'
with h5py.File(hdf5_file_name, 'w') as h5_file:
    
    #Create a group for the sign location data
    sign_group = h5_file.create_group('SignLocationData')
    
    #Store the sign location data as a single dataset- keep row-based relationships intact
    # Store the sign location data as variable-length UTF-8 strings
    dt = h5py.string_dtype(encoding='utf-8') 
    sign_location_dataset = sign_group.create_dataset('SignLocations', (len(sign_location_data_list), len(sign_location_data_list[0])), dtype=dt)

    #Populate the dataset row-by-row (to avoid storing as byte strings)
    for i, row in enumerate(sign_location_data_list):
        sign_location_dataset[i] = row  #Assign each row to the dataset
    
    #-----------------------------------METADATA INFORMATION-------------------------------------------------------
    
    #Add metadata to the group
    sign_group.attrs['source'] = 'Sign Location Data Collection by Kene Nwachukwu'
    sign_group.attrs['extraction_method'] = 'Measurement Data Collection'
    sign_group.attrs['latitude_longitude_format'] = metadata_dict['Latitude_Longitude_Format']
    sign_group.attrs['collection_tool'] = metadata_dict['Collection_Tool']
    sign_group.attrs['dataset_description'] = metadata_dict['Dataset_Description']

    #Additional metadata about the origin, conversion process, and overall data/file
    h5_file.attrs['origin'] = "Collected and Provided by Kene Nwachukwu"
    h5_file.attrs['conversion_process'] = "Converted from Excel to HDF5 with metadata extraction"
    h5_file.attrs['file_creation_date'] = '2024-10-18'
    h5_file.attrs['description'] = 'Sign location data around RPI campus collected by Kene Nwachukwu for 2023-2024'

    
    #Store the original metadata lines that Kene provided at the bottom of the Excel-
    #Create a README.md file with the metadata content
    readme_file_path = 'README.md'
    with open(readme_file_path, 'w') as f:
        f.write(f"# Sign Location Data Collection Metadata\n\n")
        f.write(f"**Dataset Description**: {metadata_dict['Dataset_Description']}\n")
        f.write(f"**Data Collection Tool**: {metadata_dict['Collection_Tool']}\n")
        f.write(f"**Latitude/Longitude Format**: {metadata_dict['Latitude_Longitude_Format']}\n")

    
    if os.path.exists(readme_file_path):
            with open(readme_file_path, 'r') as f:
                readme_content = f.read()
            h5_file.attrs['readme_markdown'] = readme_content



In [3]:
#Read the file and metadata to ensure the conversion worked properly

hdf5_file_name = 'SignLocations.h5'

#Open the HDF5 file
with h5py.File(hdf5_file_name, 'r') as h5_file:
    
    #List all groups in the file
    print("Groups in the HDF5 file:")
    for group_name in h5_file:
        print(group_name)
    
    #Access the group
    sign_group = h5_file['SignLocationData']

    #List datasets in the group
    print("\nDatasets in 'SignLocationData':")
    for dataset_name in sign_group:
        print(dataset_name)
    
    #Read the dataset 
    sign_locations_data = sign_group['SignLocations'][:]
    
    #Convert byte strings to regular strings
    decoded_data = []
    for row in sign_locations_data:
        decoded_row = [cell.decode('utf-8') if isinstance(cell, bytes) else cell for cell in row]
        decoded_data.append(decoded_row)

    print("\nDecoded Sign Locations Data (First 5 rows):")
    for row in decoded_data[:5]:
        print(row)   


   #Reading file-level metadata
    print("\nFile-level metadata:")
    for key, value in h5_file.attrs.items():
        print(f"{key}: {value}")

    #Reading group-level metadata
    print("\nGroup-level metadata (SignLocationData):")
    for key, value in sign_group.attrs.items():
        print(f"{key}: {value}")


Groups in the HDF5 file:
SignLocationData

Datasets in 'SignLocationData':
SignLocations

Decoded Sign Locations Data (First 5 rows):
['1.      ', 'Alan Computing signage', '42°  43\' 45" N', '73° 40\' 56" W', '72.58', '2024-09-13-13-30']
['2.      ', 'Map 1 ', '42° 43´ 47" N', '73° 40´ 56"W', '70.37', '2024-09-13-13-29']
['3.      ', 'Amos Eaton signpost', '42° 43´ 48" N', '73° 40´56 " W', '70.22', '2024-09-13-13-28']
['4.      ', 'Lally signpost', '42° 43´ 48" N', '73° 40´ 55" W', '71.47', '2024-09-13-13-27']
['5.      ', 'Map2 ', '42° 43´ 49" N', '73° 40´ 54" W', '70.67', '2024-09-13-13-26']

File-level metadata:
conversion_process: Converted from Excel to HDF5 with metadata extraction
description: Sign location data around RPI campus collected by Kene Nwachukwu for 2023-2024
file_creation_date: 2024-10-18
origin: Collected and Provided by Kene Nwachukwu
readme_markdown: # Sign Location Data Collection Metadata

**Dataset Description**: Dataset consists locations of maps around camp

In [9]:
#!pwd
#!git init
#!git add .
#!git commit -m "Added AIP files"
#!git remote add origin git@github.com:thernandez7/Sign_Location_data_conversion_HDF5.git
!git push -u origin master

branch 'master' set up to track 'origin/master'.


To github.com:thernandez7/Sign_Location_data_conversion_HDF5.git
   05c4ed6..21ce1d5  master -> master
